In [ ]:
## packages
import os
import random
import pickle
import numpy as np
import pandas as pd

from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages

import src.deep_models as dm 
from src.utils import get_data, scaling, get_colnames
from src.eval import eval_top_n_guides_new_genes, plot_top_n_guides_new_genes

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## set seeds
seed = 5555
random.seed(seed)
np.random.seed(seed)

## get time stamp
date = datetime.now()
date = "{}-{}-{}".format(date.year, date.month, date.day)

In [ ]:
## files and directories
date_model_trained = "2021-3-30"

input_models = "/storage/groups/haicu/workspace/crispri/models/" + date_model_trained
output_performance = "../reports/performance_eval/" + date + "/purine_genes" 
output_plots = "../reports/plots_eval/" + date + "/purine_genes/"

file_data_purine_genes = '../datasets/data_purine_gene.pickle'
file_one_hot_encoding_ML_purine_genes = '../datasets/one_hot_encoding_ML_purine_gene.pickle'
file_one_hot_encoding_DL_purine_genes = '../datasets/one_hot_encoding_DL_purine_gene.pickle'


In [ ]:
## setup parameters
time_point_of_guide_measurement = 'OD1'

n_folds_outer = 5
n_folds_inner = 5
top_n_guides_list = [1,3]

features_guide = ["guide_GC_content", "distance_start_codon", "homopolymers", "MFE_hybrid_full", "MFE_hybrid_seed", "MFE_homodimer_guide", "MFE_monomer_guide", 
               "off_target_90_100", "off_target_80_90", "off_target_70_80", "off_target_60_70"]

datasets_models_trained = ["wang_median-sub_guide", "rousset_E18_median-sub_guide", "wang_rousset_E18_median-sub_guide"]

models_trained = ["linear", "elnet", "random_forest", "GBM", "1DCNN", "GRU"]

deep_models = ["1DCNN", "GRU"]
needs_scaling = ["elnet"]

In [ ]:
## load data
data_purine_genes = get_data(file_data_purine_genes)
one_hot_encoding_ML_purine_genes = get_data(file_one_hot_encoding_ML_purine_genes)
one_hot_encoding_DL_purine_genes = get_data(file_one_hot_encoding_DL_purine_genes)

genes = data_purine_genes["geneid"].unique()

In [ ]:
%matplotlib

colnames_metrics = ['SpearmanR','Performance_Increase','Wilcoxon_p-value', 'mean_top_n_guides']

# do evaluation for each gene seperately
for gene in genes:
    print("---")
    print(gene)
    
    # save performance metrics tables for each gene
    performance_metrics_per_top_n_guides = []
    
    # create dataset for model input
    data_purine_gene_selected = data_purine_genes[data_purine_genes["geneid"] == gene].copy()
    one_hot_encoding_ML_purine_gene_selected = one_hot_encoding_ML_purine_genes[data_purine_genes["geneid"] == gene].copy()
    one_hot_encoding_DL_purine_gene_selected = one_hot_encoding_DL_purine_genes[data_purine_genes["geneid"] == gene].copy()

    X_guide_ML = pd.concat([data_purine_gene_selected[features_guide],one_hot_encoding_ML_purine_gene_selected],axis=1)
    X_guide_DL = pd.concat([data_purine_gene_selected[features_guide],one_hot_encoding_DL_purine_gene_selected],axis=1)

    log2FC_original = data_purine_gene_selected[time_point_of_guide_measurement + '_edgeR.batch'].tolist()
    
    # do evaluation for each top n guides
    for top_n_guides in top_n_guides_list:
    
        # plot output and metrics table
        filename = output_plots + "/top_" + str(top_n_guides) + "_guides_tp_" + time_point_of_guide_measurement + "_" + gene + ".pdf"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        pp = PdfPages(filename)

        performance_metrics = pd.DataFrame(columns = colnames_metrics, index = get_colnames(models_trained, datasets_models_trained))
        
        # iterate over each model and dataset
        for model in models_trained:
            print("model: " + model)

            for dataset in datasets_models_trained:
                print("dataset: " + dataset)
                predictions = np.zeros((data_purine_gene_selected.shape[0], n_folds_outer))

                for fold_outer in range(n_folds_outer):
                    
                    # prediction with DL models 
                    if model in deep_models:
                        
                        file_scaler = input_models + "/" + model + "/" + dataset + "/fold_outer_" + str(fold_outer) + "/scaler.pickle"
                        SCALE = pickle.load(open(file_scaler, 'rb'))
                        X_guide_DL_scale = scaling(SCALE, X_guide_DL, features_guide)
                
                        dir_model = input_models + "/" + model + "/" + dataset + "/fold_outer_" + str(fold_outer) + "/"
                        predictions[:,fold_outer] = dm.evaluate_deep_model_new_data(model, X_guide_DL_scale.copy(), features_guide, fold_outer, n_folds_inner, dir_model)
                    
                    # predictions with ML models
                    else:
                        
                        if model in needs_scaling:
                            file_scaler = input_models + "/" + model + "/" + dataset + "/scaler_fold_outer" + str(fold_outer) + ".pickle"
                            SCALE = pickle.load(open(file_scaler, 'rb'))
                            X_guide_ML_scale = scaling(SCALE, X_guide_ML, features_guide)
                        else:
                            X_guide_ML_scale = X_guide_ML
                            
                        file_model = input_models + "/" + model + "/" + dataset + "/model_fold_outer" + str(fold_outer) + ".pickle"
                        estimator = pickle.load(open(file_model, 'rb'))
                        predictions[:,fold_outer] = estimator.predict(X_guide_ML_scale)
                            
                predictions = pd.DataFrame({'Log2FC_original': log2FC_original, 'Log2FC_predicted': predictions.mean(axis=1)})      
                performance_metrics = eval_top_n_guides_new_genes(model, dataset, predictions, top_n_guides, performance_metrics, pp)
        
        pp.close()
        
        # save performance metrics
        filename_csv = output_performance + "/top_" + str(top_n_guides) + "_guides_tp_" + time_point_of_guide_measurement + "_" + gene + ".csv"
        os.makedirs(os.path.dirname(filename_csv), exist_ok=True)        
        performance_metrics.to_csv(filename_csv)
        
        performance_metrics_per_top_n_guides.append(performance_metrics)
        
    # plot summary histograms for each gene
    filename = output_plots + "summary_tp_" + time_point_of_guide_measurement + "_" + gene + ".pdf"
    pp = PdfPages(filename)
    plot_top_n_guides_new_genes(log2FC_original, top_n_guides_list, performance_metrics_per_top_n_guides, pp)
    pp.close()
    